In [1]:
import sqlite3
import pandas as pd
import numpy as np
import sys

In [2]:
pd.__version__

'0.24.0'

In [3]:
MQ_FEATURE_ID = 8
feature_ids = [1299, 279, 4339, 1290, 2969, 4483, 4238, 42340, 35437, 111912, 134869]

In [4]:
import pickle

# snapshot_feature_df_list = []
# for idx,feature_id in enumerate(feature_ids):
#     snapshot_feature_df_list.append((pd.read_pickle("/Users/darylwilding-mcbride/Downloads/snapshot_feature_{}_df.pickle".format(feature_id))))

with open("/Users/darylwilding-mcbride/Downloads/snapshot_mq_feature_{}.pickle".format(MQ_FEATURE_ID), "rb") as input_file:
    snapshot_feature_df_list = pickle.load(input_file)

In [5]:
len(snapshot_feature_df_list)

11

In [6]:
snapshot_feature_df_list[2].head()

,frame_id,point_id,mz,scan,intensity,peak_id,raw_frame_point,retention_time_secs
1066,41500,50229,878.400022,347,6756,0,41500|50229,4457.345016
1067,41500,50479,878.404695,348,5697,0,41500|50479,4457.345016
1068,41500,50480,878.437409,348,300,0,41500|50480,4457.345016
1069,41500,50728,878.400022,349,6756,0,41500|50728,4457.345016
1070,41500,50733,878.900134,349,3103,0,41500|50733,4457.345016


In [7]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]


In [8]:
for idx,feature_id in enumerate(feature_ids):
    df = snapshot_feature_df_list[idx]
    df["hover"] = df["mz"].map('{:,.4f} m/z'.format) + ', ' + df["scan"].map('{} scan'.format) + ', feature {}'.format(feature_id)

In [9]:
snapshot_feature_df_list[2].head()

,frame_id,point_id,mz,scan,intensity,peak_id,raw_frame_point,retention_time_secs,hover
1066,41500,50229,878.400022,347,6756,0,41500|50229,4457.345016,"878.4000 m/z, 347 scan, feature 4339"
1067,41500,50479,878.404695,348,5697,0,41500|50479,4457.345016,"878.4047 m/z, 348 scan, feature 4339"
1068,41500,50480,878.437409,348,300,0,41500|50480,4457.345016,"878.4374 m/z, 348 scan, feature 4339"
1069,41500,50728,878.400022,349,6756,0,41500|50728,4457.345016,"878.4000 m/z, 349 scan, feature 4339"
1070,41500,50733,878.900134,349,3103,0,41500|50733,4457.345016,"878.9001 m/z, 349 scan, feature 4339"


In [10]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # requires version >= 1.9.0

3.5.0


In [11]:
init_notebook_mode(connected=True)

In [16]:
import plotly.plotly as py
import plotly.graph_objs as go

traces = []
for idx,feature_id in enumerate(feature_ids):
    df = snapshot_feature_df_list[idx]
    if len(df) > 0:
        trace = go.Scatter3d(
            x = df.mz,
            y = df.scan,
            z = df.intensity,
            mode = 'markers',
            marker = dict(
                size = 5,
                color = "rgb{}".format(tableau20[idx]),
                opacity = 0.8
            ),
            text = df.hover,
            hoverinfo = 'text',
            name = "feature {}".format(feature_id)
        )
        traces.append((trace))


data = traces
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene = dict(
        xaxis = dict(
            title='m/z', autorange='reversed'),
        yaxis = dict(
            title='scan'),
        zaxis = dict(
            title='intensity')
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)
